In [7]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages.human import HumanMessage

from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.lats.node import Node
from agential.cog.lats.prompts import (
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT,
    LATS_INSTRUCTION_HOTPOTQA,
    LATS_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, 
    LATS_VALUE_INSTRUCTION_HOTPOTQA,
)
from agential.cog.lats.strategies.qa import (
    LATSAmbigNQStrategy,
    LATSFEVERStrategy,
    LATSHotQAStrategy,
    LATSTriviaQAStrategy,
    parse_qa_action,
    parse_qa_value,
)
from agential.cog.react.output import ReActOutput
from agential.utils.docstore import DocstoreExplorer


llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1.0,
    max_tokens=100,
    n=1,
    stop="."
)

/opt/anaconda3/envs/agential/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(


In [8]:
"""Test the simulate_node method."""
qa_strategy = LATSHotQAStrategy(llm=llm)

get_node_trajectory(child1)

In [11]:
from agential.cog.lats.node import Node
from agential.cog.react.output import ReActOutput
from langchain_community.chat_models.fake import FakeListChatModel
from agential.cog.lats.strategies.qa import LATSHotQAStrategy
from agential.cog.lats.prompts import HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE, LATS_VALUE_INSTRUCTION_HOTPOTQA

# Mock input parameters
question = "What is the capital of France?"
key = "Paris"
examples = HOTPOTQA_FEWSHOT_EXAMPLES_REACT
reflect_examples = HOTPOTQA_FEWSHOT_EXAMPLES_LATS_REFLECT
value_examples = HOTPOTQA_FEWSHOT_EXAMPLES_LATS_VALUE
prompt = LATS_INSTRUCTION_HOTPOTQA
reflect_prompt = LATS_REFLECT_INSTRUCTION_HOTPOTQA
value_prompt = LATS_VALUE_INSTRUCTION_HOTPOTQA
additional_keys = {}
reflect_additional_keys = {}
value_additional_keys = {}

strategy.reflection_map = [
    {
        "trajectory": "Failed trajectory",
        "reflection": "This trajectory failed because...",
    },
    {
        "trajectory": "Another failed trajectory",
        "reflection": "This trajectory failed because of something else...",
    }
]

values = strategy.evaluate_node(root, question, examples, prompt, {})


<PROMPT VALUE=====================================================================>
Analyze the trajectories of a solution to a question answering task. The trajectories are labeled by environmental observations about the situation, thoughts that can reason about the current situation and actions that can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.

Given a question and a trajectory, evaluate its correctness by focusing on the latest thought, action, and observation. Then, provide a score between 1 and 10.
Incomplete trajectories can be correct if the thoughts and actions so far are correct, even if the answer is not found yet. Do not generate additional thoughts or actions. 

Here are s

In [8]:
children_trajectories = [
    {"child_trajectory": get_node_trajectory(child), "idx": idx}
    for idx, child in enumerate(root.children)
    if not child.is_terminal
]
children_trajectories

[{'child_trajectory': '\nThought 1: Child 1', 'idx': 0},
 {'child_trajectory': '\nThought 1: Child 1.5', 'idx': 1}]

In [10]:
values

[{'explanation': 'Good trajectory.', 'value': 0.8},
 {'explanation': 'Good trajectory.', 'value': 0}]